In [1]:
import numpy as np
import pandas as pd
import cv2
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate


data = pd.read_csv('/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv')

2024-03-18 15:35:56.549494: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 15:35:56.549599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 15:35:56.677970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tqdm import tqdm

In [3]:
# Define a function to load and preprocess images in batches
def load_images_batch(image_paths, batch_size=32):
    images = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = [load_image(path) for path in batch_paths]
        images.extend(batch_images)
    return np.array(images)

# Define a function to create a data generator for masks
def mask_generator(mask_data, batch_size=32):
    num_samples = len(mask_data)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_masks = mask_data[offset:offset+batch_size]
            yield np.array(batch_masks)

def load_image(image_path):
    try:
        # Завантаження зображення за вказаним шляхом
        image = cv2.imread(image_path)
        
        # Перевірка, чи вдалося завантажити зображення
        if image is None:
            raise FileNotFoundError(f"Failed to load image at path: {image_path}")
        
        return image
        
    except Exception as e:
        # Обробка помилок
        print(f"Error loading image: {e}")
        return None

# Функція для перетворення run-length encoding у маску зображення
def rle_to_mask(rle_string, width, height):
    mask = np.zeros(width * height, dtype=np.uint8)
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    for i in range(len(rle_numbers) // 2):
        start = rle_numbers[2 * i] - 1
        length = rle_numbers[2 * i + 1]
        mask[start:start + length] = 1
    mask = mask.reshape((height, width), order='F')
    return mask

# Load data paths and preprocess images in batches
data = pd.read_csv('/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv')
image_paths = ['/kaggle/input/airbus-ship-detection/train_v2/' + row['ImageId'] for i, row in data.iterrows()]
images = load_images_batch(image_paths, batch_size=32)

# Generate masks in batches using a data generator
masks_data = [rle_to_mask(row['EncodedPixels'], width, height) if isinstance(row['EncodedPixels'], str) else np.zeros((height, width)) for i, row in data.iterrows()]
mask_gen = mask_generator(masks_data, batch_size=32)


In [ ]:
# Split data into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(image_paths, masks_data, test_size=0.2, random_state=42)


In [ ]:
# Створення Unet моделі

def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    up1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up1 = concatenate([up1, conv2], axis=3)

    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    up2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up2 = concatenate([up2, conv1], axis=3)

    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Компіляція моделі
model = build_unet(input_shape=(256, 256, 3))
model.compile(optimizer=Adam(), loss=binary_crossentropy, metrics=[dice_score])


In [ ]:
# Навчання моделі
history = model.fit(train_images, train_masks, epochs=20, batch_size=32, validation_data=(val_images, val_masks))

# Оцінка моделі
test_loss, test_dice_score = model.evaluate(test_images, test_masks)
print(f'Test Loss: {test_loss}, Test Dice Score: {test_dice_score}')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU

# Step 1: Data Preprocessing
train_df = pd.read_csv("/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv")

# Convert run-length encoded masks to binary masks
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction



In [ ]:
# Implement UNet architecture
def unet_model(input_shape=(768, 768, 3)):
    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up4 = tf.keras.layers.concatenate([up4, conv2], axis=3)
    conv4 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up5 = tf.keras.layers.concatenate([up5, conv1], axis=3)
    conv5 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Step 3: Training
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2)

# Define loss function (e.g., Dice Loss) and metrics (e.g., Dice Score)
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Compile the model
model = unet_model()
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])

# Train the model
model.fit(train_images, train_masks, validation_data=(val_images, val_masks), epochs=epochs, batch_size=batch_size)

# Step 4: Evaluation
# Evaluate the model on the validation set
val_loss, val_dice = model.evaluate(val_images, val_masks)

# Step 5: Inference
# Perform inference on the test set and generate predictions
test_predictions = model.predict(test_images)
